# Data Visualization 1 Assignment - Leon Tan

For this assignment, I explored three ways to fetch stock information from the Yahoo Finance website, Rapid API's Yahoo Finance API, and Alpha Vantage's API. Additionally, given the visualization requirement of this assignment, I used two methods to display the findings at the end using Plotly's functions. 

As a requirement, the user must be given the choice of choosing from 10 stocks of our choosing, select between dates from 1 January 2010 and 31 December 2017, select the method of RSI calculation using EMWA or SMA, and specify the moving period for their selection. 

With the requirements as a guideline for the user interface, the majority of the code will be run in the same cell, while I've also included various functions in other cells to aid in the project. I'm also attempting to use try and except blocks to account for user error and ensure that the program runs as smoothly as possible. 

However, given the nature of the user interface, I won't be able to show step-by-step processes to clean the data or visualize outliers, to name a few; I did all this in a separate notebook and have written the necessary code to take care of discrepancies.

You'll find that the code for the user interface will remain largely the same throughout the next three segments, barring slight tweaks to how I fetch data from the three APIs and certain functions I wrote to help with calling the APIs. The visualization methods are both from Plotly and vary slightly on aesthetics. Further comments will be written in the following cells to justify my rationale for the code I've written. All API keys have been removed for security purposes and as per assignment requirements. 

Answer 1: Using Yahoo Finance website's API and Plotly's Figure object functions

In [1]:
import time
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import requests
import pandas_market_calendars as mcal 
import plotly.graph_objects as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
import cufflinks as cf
init_notebook_mode(connected=True) 
cf.go_offline()

In the following function, I've written it to accept four parameters - the moving window period, method for calculation, a variable number of dataframe arguments, and a key-worded argument for the final stock list. I will be creating a new dataframe here to store the RSI data for each of the dataframes passed in. 

Testing all the stock dates in a separate notebook, I've found the date values to all be equal using the df.equals() function, and have thus decided to set the new dataframe's date using the date column of any of the dataframes appended during the user input. 

Using a for loop to iterate over the tuple of dataframes passed in, the RSI column will be created for each of the dataframes passed in after calculation has been completed, and a new column in the empty dataframe will be created each time a loop completes. To automate the naming of the columns, I've made use of the index position for the sole key-worded argument's list items, and utilized a counter variable to iterate over the list's indices for each loop.

Following that, I'd conducted outlier checks using plt.boxplot() on a separate notebook and have included an additional feature to omit the outliers of the new dataframe from the final chart. Once the outliers have been dropped, the plotly.go.Figure() object created initially will include the various trace data which will be added via a for loop. Once the process has been completed, the chart will be return on the function call.  

In [3]:
def RSI_calculator(window=14,method="SMA",*df,**stocks):
    
    fig = go.Figure()
    
    new_df = pd.DataFrame()
    new_df["Date"] = df[0]["Date"]
    
    index_counter = 0
    
    for i in df:
        delta = i["Close"].diff()

        up,down = delta.copy(),delta.copy()
        up[up<0] = 0
        down[down>0] = 0

        if method == "SMA": 
            gain = up.rolling(window).mean()
            loss = down.abs().rolling(window).mean()
        else:
            gain = up.ewm(span=window).mean()
            loss = down.abs().ewm(span=window).mean()

        RSI = gain/loss
        i["RSI"] = pd.Series(100 - (100/(1+RSI)),name="RSI")
        i["RSI"].fillna(0,inplace=True)
        
        new_df[stocks["fin"][index_counter]] = i["RSI"]
        
        index_counter += 1
        
    Q_1 = new_df[stocks["fin"]].quantile(0.25)
    Q_3 = new_df[stocks["fin"]].quantile(0.75)

    I_Q_R = Q_3-Q_1
    
    new_df.set_index("Date",inplace=True)
    
    new_df = new_df[~((new_df[stocks["fin"]] < (Q_1 - 1.5 * I_Q_R)) |(new_df[stocks["fin"]] > (Q_3 + 1.5 * I_Q_R))).any(axis=1)]
    
    for i in stocks["fin"]:
        fig.add_trace(go.Scatter(x=new_df.index, y=new_df[i], name=i))
                
    return fig.show()

In the user interface below, I've made use of two counter variables to keep the main while loop in check, while also including further while loops to handle exceptions. The main while loop will execute as long as the number provided by the user does not exceed the number of default stocks and must not be less than 1, and appends the user's stock choices into an empty list. I have used the default stock list to compare for duplicate names, and will only display unchosen stock on subsequent loops. Once the user has reached the number of stocks they've specified, the code will then ask for the four preferences, and will execute the for loop to fetch the stock data from Yahoo's website if all input is accurate. Upon completion of the API call, we'll then call the function to display the chart based on the user's preferences. 

The stock list in the variable stock was provided as the default list by the instructor, but please note that after testing for dates separately, I've found that FB and TSLA only IPO-ed after 1 January 2010. For the purpose of this method, I've tried to replicate the instructor's sample output as closely as possible, which includes use of the default stock list. Using a for loop to fetch the data, I appended each dataframe into an empty list variable for unpacking in the function call later on. 

Additionally, to ensure that the user sticks to the "YYYY-MM-dd" format as a requirement, I converted the string into a datetime object, and subsequently a formatted string object, before converting it into a timestamp using the datetime module - an alternative would be to use the pandas to_datetime() function. 

Finally, just to note that the Open, Close, High, Low and Volume fetched by this website seems to have been adjusted, with splits accounted for. In Alpha Vantage's API, I'd noticed that the five columns were raw data instead. 

In [4]:
stock = ["FB", "AAPL", "AMZN", "IBM", "GOOGL", "MSFT", "NAV", "O", "QCOM", "TSLA"]
counter = 0
stock_list = []
nl = '\n'
stock_number = 1
dfs = []

while True:
    in_1 = input("Please enter number of stocks: ")
    print()
    
    try:
        if in_1.isnumeric() == False:
            raise ValueError
        elif int(in_1) > len(stock) or int(in_1) < 1:
            raise ValueError
        else:
            in_1 = int(in_1)
            break

    except ValueError:
        print("Stock number is invalid.")
        print()

while counter < len(stock): 
    if in_1 < len(stock):
        try:
            print(f"Displaying stocks to be selected: {nl}{nl.join([i for i in stock if i not in stock_list])}")
            print()
            in_2 = input(f"Please choose stock number {stock_number}: ").upper()
            print()
            
            if in_2 not in stock:
                raise ValueError
        
            stock_list.append(in_2)
            counter += 1

            if stock_number == in_1:
                print(f"Displaying remaining unselected stock: {nl}{nl.join([i for i in stock if i not in stock_list])}")
                print()
                
                while True: 
                    try:
                        pref_1 = input("Please advise on RSI Calculation Method (EMWA or SMA): ").upper()
                        
                        if pref_1 != "EMWA" and pref_1 != "SMA":
                            raise ValueError
                            
                        else:
                            break
            
                    except ValueError:
                        print("Please only key in EMWA or SMA")
                    
                while True:
                    try:
                        pref_2 = input("Please enter Moving Window Range: ")
                        
                        if pref_2.isnumeric() == False:
                            raise ValueError
                        
                        else:
                            pref_2 = int(pref_2)
                            break
                    
                    except ValueError:
                        print("Please enter a numerical value.")
                
                while True:                 
                    pref_3 = input("Please enter start date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: ")

                    try:
                        if datetime.strptime(pref_3,"%Y-%m-%d") < datetime.strptime("2010-01-01","%Y-%m-%d") or datetime.strptime(pref_3,"%Y-%m-%d") >= datetime.strptime("2017-12-31","%Y-%m-%d"):
                            raise ValueError
                        
                        elif pref_3 != datetime.strptime(pref_3,"%Y-%m-%d").strftime("%Y-%m-%d"):
                            raise ValueError
                            
                        else:
                            dt1 = datetime.strptime(pref_3,"%Y-%m-%d").strftime("%Y-%m-%d")
                            start_date1 = int(round(datetime.strptime(dt1,"%Y-%m-%d").timestamp()))
                            break

                    except ValueError:
                        print("Incorrect date time format or date. Please enter valid date in YYYY-MM-DD format.")
                    
                while True:
                    pref_4 = input("Please enter end date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: ") 
                           
                    try: 
                        if datetime.strptime(pref_4,"%Y-%m-%d") <= datetime.strptime(pref_3,"%Y-%m-%d") or datetime.strptime(pref_4,"%Y-%m-%d") > datetime.strptime("2017-12-31","%Y-%m-%d"):
                            raise ValueError
                        
                        elif pref_4 != datetime.strptime(pref_4,"%Y-%m-%d").strftime("%Y-%m-%d"):
                            raise ValueError
                        
                        else:
                            dt2 = datetime.strptime(pref_4,"%Y-%m-%d").strftime("%Y-%m-%d")
                            end_date1 = int(round(datetime.strptime(dt2,"%Y-%m-%d").timestamp()))
                            break
                    
                    except ValueError:
                        print("Incorrect date time format or date. Please enter valid date in YYYY-MM-DD format.")

                for i in stock_list: 
                    df = pd.read_csv(
                            f"https://query1.finance.yahoo.com/v7/finance/download/{i}?period1={start_date1}&period2={end_date1}&interval=1d&events=history&includeAdjustedClose=true")
                    dfs.append(df)

                RSI_calculator(pref_2,pref_1,*dfs,fin=stock_list)

                counter = len(stock)
                break
                
            stock_number += 1 

        except ValueError:
            print("Stock name is invalid. Please try again.")
            print()
        
    else:
        print(f"Displaying all stocks to be calculated: {nl}{nl.join(stock)}")
        print()
        stock_list = stock
        
        while True: 
            try:
                pref_1 = input("Please advise on RSI Calculation Method (EMWA or SMA): ").upper()
                
                if pref_1 != "EMWA" and pref_1 != "SMA":
                    raise ValueError
                    
                else:    
                    break

            except ValueError:
                print("Please only key in EMWA or SMA")
        
        while True:
            try:
                pref_2 = input("Please enter Moving Window Range: ")

                if pref_2.isnumeric() == False:
                    raise ValueError

                else:
                    pref_2 = int(pref_2)
                    break

            except ValueError:
                print("Please enter a numerical value.")

        while True:                 
            pref_3 = input("Please enter start date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: ")

            try:
                if datetime.strptime(pref_3,"%Y-%m-%d") < datetime.strptime("2010-01-01","%Y-%m-%d") or datetime.strptime(pref_3,"%Y-%m-%d") >= datetime.strptime("2017-12-31","%Y-%m-%d"):
                    raise ValueError

                elif pref_3 != datetime.strptime(pref_3,"%Y-%m-%d").strftime("%Y-%m-%d"):
                    raise ValueError

                else:
                    dt1 = datetime.strptime(pref_3,"%Y-%m-%d").strftime("%Y-%m-%d")
                    start_date1 = int(round(datetime.strptime(dt1,"%Y-%m-%d").timestamp()))
                    break

            except ValueError:
                print("Incorrect date time format or date. Please enter valid date in YYYY-MM-DD format.")

        while True:
            pref_4 = input("Please enter end date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: ") 

            try: 
                if datetime.strptime(pref_4,"%Y-%m-%d") <= datetime.strptime(pref_3,"%Y-%m-%d") or datetime.strptime(pref_4,"%Y-%m-%d") > datetime.strptime("2017-12-31","%Y-%m-%d"):
                    raise ValueError

                elif pref_4 != datetime.strptime(pref_4,"%Y-%m-%d").strftime("%Y-%m-%d"):
                    raise ValueError

                else:
                    dt2 = datetime.strptime(pref_4,"%Y-%m-%d").strftime("%Y-%m-%d")
                    end_date1 = int(round(datetime.strptime(dt2,"%Y-%m-%d").timestamp()))
                    break

            except ValueError:
                print("Incorrect date time format or date. Please enter valid date in YYYY-MM-DD format.")

        for i in stock_list: 
            df = pd.read_csv(
                    f"https://query1.finance.yahoo.com/v7/finance/download/{i}?period1={start_date1}&period2={end_date1}&interval=1d&events=history&includeAdjustedClose=true")
            dfs.append(df)

        RSI_calculator(pref_2,pref_1,*dfs,fin=stock_list)

        counter = len(stock)
        stock_number = len(stock)
        
        break

Please enter number of stocks: 5

Displaying stocks to be selected: 
FB
AAPL
AMZN
IBM
GOOGL
MSFT
NAV
O
QCOM
TSLA

Please choose stock number 1: FB

Displaying stocks to be selected: 
AAPL
AMZN
IBM
GOOGL
MSFT
NAV
O
QCOM
TSLA

Please choose stock number 2: AAPL

Displaying stocks to be selected: 
AMZN
IBM
GOOGL
MSFT
NAV
O
QCOM
TSLA

Please choose stock number 3: GOOGL

Displaying stocks to be selected: 
AMZN
IBM
MSFT
NAV
O
QCOM
TSLA

Please choose stock number 4: msft

Displaying stocks to be selected: 
AMZN
IBM
NAV
O
QCOM
TSLA

Please choose stock number 5: tsla

Displaying remaining unselected stock: 
AMZN
IBM
NAV
O
QCOM

Please advise on RSI Calculation Method (EMWA or SMA): emwa
Please enter Moving Window Range: 14
Please enter start date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: 2014-12-31
Please enter end date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: 2017-12-31


Answer 2: Using RapidAPI's Yahoo Finance API

In this method, I defined a function which will help convert the JSON data downloaded from the API into individual dataframes to be appended in a list of dataframes; similar to the previous answer's method. In this converter function, I've renamed the columns to reflect those from the columns fetched by the website, and upon analysis in a separate notebook, I've found that the data fetched by RapidAPI contains duplicate columns which stores dividend, type and amount information, while having null information for everything else. Knowing this, I've also set the function to drop the unnecessary columns by finding the null values that are in the Open, High, Low, Close, Volume and Adjusted Close columns. 

Furthermore, the dates fetched were stored in timestamps instead, and I've included code below to ensure that the correct datetime format is passed in. 

In [5]:
def API_fetcher(json):
    df2 = pd.DataFrame(json["prices"])
    df2["date"] = pd.to_datetime(df2["date"],unit="s").dt.date
    df2.sort_values("date",inplace=True)
    df2.dropna(subset=["open","high","low","close","volume","adjclose"],inplace=True)
    df2.reset_index(inplace=True, drop=True)
    df2.rename(columns={"date":"Date","open":"Open","high":"High","low":"Low","close":"Close","volume":"Volume","adjclose":"Adj Close"},inplace=True)

    dfs1.append(df2)

The RSI_calculator function makes its return here with minor differences. In this version, I used the pandas market calendar package to generate my date column in the new dataframe. Given that Yahoo Finance's API uses a similar timezone as the NYSE's, the range generated would be similar to the dates converted from timestamps in the dataframes. 

This time round, I included all outliers in case the user would like to see the full data range. Here, I'll work under the assumption that NaN values are to be considered as 0, and the result will reflect the full range of data available in the RSI columns. 

Similar to the previous answer, I used Plotly's Figure object to increment traces using a for loop. The sample output here shows what'd happen if the user decides to ask for RSI information for all 10 stocks. 

The 10 stocks used are largely similar to the default ones, with the exception of Netflix and Nvidia, which I replaced Facebook and Tesla with given their later IPO dates.  

In [6]:
def RSI_calculator2(window2=14,method2="SMA",*df2,**stocks2):
    
    fig2 = go.Figure()
    new_df2 = pd.DataFrame()
  
    nyse = mcal.get_calendar("NYSE")
    date_range = nyse.schedule(start_date=datetime.utcfromtimestamp(start_date2).strftime("%Y-%m-%d"), end_date=datetime.utcfromtimestamp(end_date2).strftime("%Y-%m-%d"))
    new_df2["Date"] = mcal.date_range(date_range, frequency="1D")
    new_df2["Date"] = new_df2["Date"].dt.date
    
    index_counter = 0
    
    for i in df2:
        delta = i["Close"].diff()

        up,down = delta.copy(),delta.copy()
        up[up<0] = 0
        down[down>0] = 0

        if method2 == "SMA": 
            gain = up.rolling(window2).mean()
            loss = down.abs().rolling(window2).mean()
        else:
            gain = up.ewm(span=window2).mean()
            loss = down.abs().ewm(span=window2).mean()

        RSI = gain/loss
        i["RSI"] = pd.Series(100 - (100/(1+RSI)),name="RSI")
        i["RSI"].fillna(0,inplace=True)
        
        new_df2[stocks2["fin2"][index_counter]] = i["RSI"]

        index_counter += 1
        
    new_df2.set_index("Date",inplace=True)   
    
    for i in stocks2["fin2"]:
        fig2.add_trace(go.Scatter(x=new_df2.index, y=new_df2[i], name=i))
   
    return fig2.show()

In [7]:
stock1 = ["NFLX", "AAPL", "AMZN", "IBM", "GOOGL", "MSFT", "NAV", "O", "QCOM", "NVDA"]
counter1 = 0
stock_list1 = []
stock_number1 = 1
dfs1 = []

while True:
    input_1 = input("Please enter number of stocks: ")
    print()
    
    try:
        if input_1.isnumeric() == False:
            raise ValueError
        elif int(input_1) > len(stock1) or int(input_1) < 1:
            raise ValueError
        else:
            input_1 = int(input_1)
            break

    except ValueError:
        print("Stock number is invalid.")
        print()

while counter1 < len(stock1): 
    if input_1 < len(stock1):
        try:
            print(f"Displaying stocks to be selected: {nl}{nl.join([i for i in stock1 if i not in stock_list1])}")
            print()
            input_2 = input(f"Please choose stock number {stock_number1}: ").upper()
            print()
            
            if input_2 not in stock1:
                raise ValueError
        
            stock_list1.append(input_2)
            counter1 += 1

            if stock_number1 == input_1:
                print(f"Displaying remaining unselected stock: {nl}{nl.join([i for i in stock1 if i not in stock_list1])}")
                print()
                
                while True: 
                    try:
                        preference_1 = input("Please advise on RSI Calculation Method (EMWA or SMA): ").upper()
                        
                        if preference_1 != "EMWA" and preference_1 != "SMA":
                            raise ValueError
                            
                        else:
                            break
            
                    except ValueError:
                        print("Please only key in EMWA or SMA")
                    
                while True:
                    try:
                        preference_2 = input("Please enter Moving Window Range: ")
                        
                        if preference_2.isnumeric() == False:
                            raise ValueError
                        
                        else:
                            preference_2 = int(preference_2)
                            break
                    
                    except ValueError:
                        print("Please enter a numerical value.")
                
                while True:                 
                    preference_3 = input("Please enter start date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: ")

                    try:
                        if datetime.strptime(preference_3,"%Y-%m-%d") < datetime.strptime("2010-01-01","%Y-%m-%d") or datetime.strptime(preference_3,"%Y-%m-%d") >= datetime.strptime("2017-12-31","%Y-%m-%d"):
                            raise ValueError
                        
                        elif preference_3 != datetime.strptime(preference_3,"%Y-%m-%d").strftime("%Y-%m-%d"):
                            raise ValueError
                            
                        else:
                            dt3 = datetime.strptime(preference_3,"%Y-%m-%d").strftime("%Y-%m-%d")
                            start_date2 = int(round(datetime.strptime(dt3,"%Y-%m-%d").timestamp()))
                            break

                    except ValueError:
                        print("Incorrect date time format or date. Please enter valid date in YYYY-MM-DD format.")
                    
                while True:
                    preference_4 = input("Please enter end date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: ") 
                           
                    try: 
                        if datetime.strptime(preference_4,"%Y-%m-%d") <= datetime.strptime(preference_3,"%Y-%m-%d") or datetime.strptime(preference_4,"%Y-%m-%d") > datetime.strptime("2017-12-31","%Y-%m-%d"):
                            raise ValueError
                        
                        elif preference_4 != datetime.strptime(preference_4,"%Y-%m-%d").strftime("%Y-%m-%d"):
                            raise ValueError
                        
                        else:
                            dt4 = datetime.strptime(preference_4,"%Y-%m-%d").strftime("%Y-%m-%d")
                            end_date2 = int(round(datetime.strptime(dt4,"%Y-%m-%d").timestamp()))
                            break
                    
                    except ValueError:
                        print("Incorrect date time format or date. Please enter valid date in YYYY-MM-DD format.")

                for i in stock_list1: 
                    
                    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-historical-data"

                    querystring = {"period1":start_date2,"period2":end_date2,"symbol":i,"frequency":"1d","filter":"history"}

                    headers = {
                        'x-rapidapi-key': "<insert API key>",
                        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
                        }

                    response = requests.request("GET", url, headers=headers, params=querystring)
                    data = response.json()
                    
                    API_fetcher(data)

                RSI_calculator2(preference_2,preference_1,*dfs1,fin2=stock_list1)

                counter1 = len(stock1)
                break
                
            stock_number1 += 1 

        except ValueError:
            print("Stock name is invalid. Please try again.")
            print()
        
    else:
        print(f"Displaying all stocks to be calculated: {nl}{nl.join(stock)}")
        print()
        stock_list1 = stock1
        
        while True: 
            try:
                preference_1 = input("Please advise on RSI Calculation Method (EMWA or SMA): ").upper()
                
                if preference_1 != "EMWA" and preference_1 != "SMA":
                    raise ValueError
                    
                else:    
                    break

            except ValueError:
                print("Please only key in EMWA or SMA")
        
        while True:
            try:
                preference_2 = input("Please enter Moving Window Range: ")

                if preference_2.isnumeric() == False:
                    raise ValueError

                else:
                    preference_2 = int(preference_2)
                    break

            except ValueError:
                print("Please enter a numerical value.")

        while True:                 
            preference_3 = input("Please enter start date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: ")

            try:
                if datetime.strptime(preference_3,"%Y-%m-%d") < datetime.strptime("2010-01-01","%Y-%m-%d") or datetime.strptime(preference_3,"%Y-%m-%d") >= datetime.strptime("2017-12-31","%Y-%m-%d"):
                    raise ValueError

                elif preference_3 != datetime.strptime(preference_3,"%Y-%m-%d").strftime("%Y-%m-%d"):
                    raise ValueError

                else:
                    dt3 = datetime.strptime(preference_3,"%Y-%m-%d").strftime("%Y-%m-%d")
                    start_date2 = int(round(datetime.strptime(dt3,"%Y-%m-%d").timestamp()))
                    break

            except ValueError:
                print("Incorrect date time format or date. Please enter valid date in YYYY-MM-DD format.")

        while True:
            preference_4 = input("Please enter end date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: ") 

            try: 
                if datetime.strptime(preference_4,"%Y-%m-%d") <= datetime.strptime(preference_3,"%Y-%m-%d") or datetime.strptime(preference_4,"%Y-%m-%d") > datetime.strptime("2017-12-31","%Y-%m-%d"):
                    raise ValueError

                elif preference_4 != datetime.strptime(preference_4,"%Y-%m-%d").strftime("%Y-%m-%d"):
                    raise ValueError

                else:
                    dt4 = datetime.strptime(preference_4,"%Y-%m-%d").strftime("%Y-%m-%d")
                    end_date2 = int(round(datetime.strptime(dt4,"%Y-%m-%d").timestamp()))
                    break

            except ValueError:
                print("Incorrect date time format or date. Please enter valid date in YYYY-MM-DD format.")

        for i in stock_list1: 

            url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-historical-data"

            querystring = {"period1":start_date2,"period2":end_date2,"symbol":i,"frequency":"1d","filter":"history"}

            headers = {
                'x-rapidapi-key': "<insert API key>",
                'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
                }
            
            response1 = requests.request("GET", url, headers=headers, params=querystring)
            data1 = response1.json()
            
            API_fetcher(data1)

        RSI_calculator2(preference_2,preference_1,*dfs1,fin2=stock_list1)

        counter1 = len(stock1)
        stock_number1 = len(stock1)
        
        break

Please enter number of stocks: 10

Displaying all stocks to be calculated: 
FB
AAPL
AMZN
IBM
GOOGL
MSFT
NAV
O
QCOM
TSLA

Please advise on RSI Calculation Method (EMWA or SMA): emwa
Please enter Moving Window Range: 14
Please enter start date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: 2014-12-31
Please enter end date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: 2017-12-31


Answer 3: Using Alpha Vantage's API and Plotly's iplot() function

In this answer, I made use of the Alpha Vantage package which is available in Python, and have created a function which will help with fetching data to store in dataframes.

Alpha Vantage's API fetches raw stock data instead of adjustments, so I proceeded to use the raw close data for more authentic results. Alternatively, I could've used the adjusted close data to calculate RSI with. 

Additionally, Alpha Vantage uses the local timezone by default, which means that the timestamps specified using Python's datetime module will be one day later than the ones from Yahoo Finance and RapidAPI. To overcome this, I converted the user's preferred dates into timestamps based on the UTC timezone instead. 

Furthermore, free users of Alpha Vantage's API are only allowed 5 requests per minute. I've overcome this by setting a delay of 1 minute using the time.sleep() function, when the stock list index is a multiple of 5 and is not 0. 

In the calculation function, I plotted the chart using Plotly's iplot() function, and have excluded all outliers from the final visualization. 

In [8]:
from alpha_vantage.timeseries import TimeSeries

In [9]:
def get_daily_adj(stock, start_dt = None, end_dt=None):
    ts = TimeSeries(key="<insert API key>",output_format="pandas")
    data2,metadata = ts.get_daily_adjusted(symbol=stock, outputsize="full")
    data2.sort_index(inplace=True)
    
    if start_dt != None and end_dt != None:
        data3 = data2[data2.index >= start_dt]
        data3 = data3[data3.index <= end_dt]
        dfs2.append(data3)

    elif start_dt != None or end_dt != None:
        if end_dt == None:
            data4 = data2[data2.index >= start_dt]
        else:
            data4 = data2[data2.index <= end_dt]
        dfs2.append(data4)
    else:
        dfs2.append(data2)

In [10]:
def RSI_calculator3(window3=14,method3="SMA",*df3,**stocks3):
    
    new_df3 = pd.DataFrame()
  
    nyse = mcal.get_calendar("NYSE")
    date_range1 = nyse.schedule(start_date=datetime.utcfromtimestamp(start_date3).strftime("%Y-%m-%d"), end_date=datetime.utcfromtimestamp(end_date3).strftime("%Y-%m-%d"))
    new_df3["Date"] = mcal.date_range(date_range1, frequency="1D")
    new_df3["Date"] = new_df3["Date"].dt.date
    new_df3.set_index("Date",inplace=True)

    index_counter = 0
    
    for i in df3:
        delta = i["4. close"].diff()

        up,down = delta.copy(),delta.copy()
        up[up<0] = 0
        down[down>0] = 0

        if method3 == "SMA": 
            gain = up.rolling(window3).mean()
            loss = down.abs().rolling(window3).mean()
        else:
            gain = up.ewm(span=window3).mean()
            loss = down.abs().ewm(span=window3).mean()

        RSI = gain/loss
        i["RSI"] = pd.Series(100 - (100/(1+RSI)),name="RSI")
        i["RSI"].fillna(0,inplace=True)
       
        new_df3[stocks3["fin3"][index_counter]] = i["RSI"]
        index_counter += 1
            
    q1 = new_df3[stocks3["fin3"]].quantile(0.25)
    q3 = new_df3[stocks3["fin3"]].quantile(0.75)

    iqr = q3-q1
        
    new_df3 = new_df3[~((new_df3[stocks3["fin3"]] < (q1 - 1.5 * iqr)) |(new_df3[stocks3["fin3"]] > (q3 + 1.5 * iqr))).any(axis=1)]
        
    return new_df3.iplot()

In [11]:
stock2 = ["NFLX", "AAPL", "AMZN", "IBM", "GOOGL", "MSFT", "NAV", "O", "QCOM", "NVDA"]
counter2 = 0
stock_list2 = []
stock_number2 = 1
dfs2 = []

while True:
    stock_amt = input("Please enter number of stocks: ")
    print()
    
    try:
        if stock_amt.isnumeric() == False:
            raise ValueError
        elif int(stock_amt) > len(stock2) or int(stock_amt) < 1:
            raise ValueError
        else:
            stock_amt = int(stock_amt)
            break

    except ValueError:
        print("Stock number is invalid.")
        print()

while counter2 < len(stock2): 
    if stock_amt < len(stock2):
        try:
            print(f"Displaying stocks to be selected: {nl}{nl.join([i for i in stock2 if i not in stock_list2])}")
            print()
            stock_name = input(f"Please choose stock number {stock_number2}: ").upper()
            print()
            
            if stock_name not in stock2:
                raise ValueError
        
            stock_list2.append(stock_name)
            counter2 += 1

            if stock_number2 == stock_amt:
                print(f"Displaying remaining unselected stock: {nl}{nl.join([i for i in stock2 if i not in stock_list2])}")
                print()
                
                while True: 
                    try:
                        choice_1 = input("Please advise on RSI Calculation Method (EMWA or SMA): ").upper()
                        
                        if choice_1 != "EMWA" and choice_1 != "SMA":
                            raise ValueError
                            
                        else:
                            break
            
                    except ValueError:
                        print("Please only key in EMWA or SMA")
                    
                while True:
                    try:
                        choice_2 = input("Please enter Moving Window Range: ")
                        
                        if choice_2.isnumeric() == False:
                            raise ValueError
                        
                        else:
                            choice_2 = int(choice_2)
                            break
                    
                    except ValueError:
                        print("Please enter a numerical value.")
                
                while True:                 
                    choice_3 = input("Please enter start date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: ")

                    try:
                        if datetime.strptime(choice_3,"%Y-%m-%d") < datetime.strptime("2010-01-01","%Y-%m-%d") or datetime.strptime(choice_3,"%Y-%m-%d") >= datetime.strptime("2017-12-31","%Y-%m-%d"):
                            raise ValueError
                        
                        elif choice_3 != datetime.strptime(choice_3,"%Y-%m-%d").strftime("%Y-%m-%d"):
                            raise ValueError
                            
                        else:
                            dt5 = datetime.strptime(choice_3,"%Y-%m-%d").strftime("%Y-%m-%d")
                            start_date3 = int(round(datetime.strptime(dt5,"%Y-%m-%d").timestamp()))
                            utc_start = datetime.utcfromtimestamp(start_date3).strftime("%Y-%m-%d")
                            break

                    except ValueError:
                        print("Incorrect date time format or date. Please enter valid date in YYYY-MM-DD format.")
                    
                while True:
                    choice_4 = input("Please enter end date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: ") 
                           
                    try: 
                        if datetime.strptime(choice_4,"%Y-%m-%d") <= datetime.strptime(choice_3,"%Y-%m-%d") or datetime.strptime(choice_4,"%Y-%m-%d") > datetime.strptime("2017-12-31","%Y-%m-%d"):
                            raise ValueError
                        
                        elif choice_4 != datetime.strptime(choice_4,"%Y-%m-%d").strftime("%Y-%m-%d"):
                            raise ValueError
                        
                        else:
                            dt6 = datetime.strptime(choice_4,"%Y-%m-%d").strftime("%Y-%m-%d")
                            end_date3 = int(round(datetime.strptime(dt6,"%Y-%m-%d").timestamp()))
                            utc_end = datetime.utcfromtimestamp(end_date3).strftime("%Y-%m-%d")
                            break
                    
                    except ValueError:
                        print("Incorrect date time format or date. Please enter valid date in YYYY-MM-DD format.")

                for i in stock_list2:
                    if stock_list2.index(i) % 5 == 0 and stock_list2.index(i) !=0:
                        time.sleep(60)
                    else:
                        get_daily_adj(i,start_dt=utc_start,end_dt=utc_end)

                RSI_calculator3(choice_2,choice_1,*dfs2,fin3=stock_list2)

                counter2 = len(stock2)
                break
                
            stock_number2 += 1 

        except ValueError:
            print("Stock name is invalid. Please try again.")
            print()
        
    else:
        print(f"Displaying all stocks to be calculated: {nl}{nl.join(stock2)}")
        print()
        stock_list2 = stock2
        
        while True: 
            try:
                choice_1 = input("Please advise on RSI Calculation Method (EMWA or SMA): ").upper()
                
                if choice_1 != "EMWA" and choice_1 != "SMA":
                    raise ValueError
                    
                else:    
                    break

            except ValueError:
                print("Please only key in EMWA or SMA")
        
        while True:
            try:
                choice_2 = input("Please enter Moving Window Range: ")

                if choice_2.isnumeric() == False:
                    raise ValueError

                else:
                    choice_2 = int(choice_2)
                    break

            except ValueError:
                print("Please enter a numerical value.")

        while True:                 
            choice_3 = input("Please enter start date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: ")

            try:
                if datetime.strptime(choice_3,"%Y-%m-%d") < datetime.strptime("2010-01-01","%Y-%m-%d") or datetime.strptime(choice_3,"%Y-%m-%d") >= datetime.strptime("2017-12-31","%Y-%m-%d"):
                    raise ValueError

                elif choice_3 != datetime.strptime(choice_3,"%Y-%m-%d").strftime("%Y-%m-%d"):
                    raise ValueError

                else:
                    dt5 = datetime.strptime(choice_3,"%Y-%m-%d").strftime("%Y-%m-%d")
                    start_date3 = int(round(datetime.strptime(dt5,"%Y-%m-%d").timestamp()))
                    utc_start = datetime.utcfromtimestamp(start_date3).strftime("%Y-%m-%d")
                    break

            except ValueError:
                print("Incorrect date time format or date. Please enter valid date in YYYY-MM-DD format.")

        while True:
            choice_4 = input("Please enter end date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: ") 

            try: 
                if datetime.strptime(choice_4,"%Y-%m-%d") <= datetime.strptime(choice_3,"%Y-%m-%d") or datetime.strptime(choice_4,"%Y-%m-%d") > datetime.strptime("2017-12-31","%Y-%m-%d"):
                    raise ValueError

                elif choice_4 != datetime.strptime(choice_4,"%Y-%m-%d").strftime("%Y-%m-%d"):
                    raise ValueError

                else:
                    dt6 = datetime.strptime(choice_4,"%Y-%m-%d").strftime("%Y-%m-%d")
                    end_date3 = int(round(datetime.strptime(dt6,"%Y-%m-%d").timestamp()))
                    utc_end = datetime.utcfromtimestamp(end_date3).strftime("%Y-%m-%d")
                    break

            except ValueError:
                print("Incorrect date time format or date. Please enter valid date in YYYY-MM-DD format.")

        for i in stock_list2:
            if stock_list2.index(i) % 5 == 0 and stock_list2.index(i) != 0:
                time.sleep(60)
            else:
                get_daily_adj(i,start_dt=utc_start,end_dt=utc_end)

        RSI_calculator3(choice_2,choice_1,*dfs2,fin3=stock_list2)

        counter2 = len(stock2)
        stock_number2 = len(stock2)
        
        break

Please enter number of stocks: 5

Displaying stocks to be selected: 
NFLX
AAPL
AMZN
IBM
GOOGL
MSFT
NAV
O
QCOM
NVDA

Please choose stock number 1: NFLX

Displaying stocks to be selected: 
AAPL
AMZN
IBM
GOOGL
MSFT
NAV
O
QCOM
NVDA

Please choose stock number 2: aapl

Displaying stocks to be selected: 
AMZN
IBM
GOOGL
MSFT
NAV
O
QCOM
NVDA

Please choose stock number 3: googl

Displaying stocks to be selected: 
AMZN
IBM
MSFT
NAV
O
QCOM
NVDA

Please choose stock number 4: ibm

Displaying stocks to be selected: 
AMZN
MSFT
NAV
O
QCOM
NVDA

Please choose stock number 5: nvda

Displaying remaining unselected stock: 
AMZN
MSFT
NAV
O
QCOM

Please advise on RSI Calculation Method (EMWA or SMA): sma
Please enter Moving Window Range: 14
Please enter start date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: 2014-12-31
Please enter end date between 2010-01-01 and 2017-12-31 in YYYY-MM-DD format: 2017-12-31
